[문제] Managing Conversation History
- trim_meassage()
- RunnablePassthrough
- itmegetter()

세션ID 설정하여, 대화 여러번 주고 받아야 함. 추가로 대화를 주고 받은 내역 출력.
LLM 모델이 과거 대화를 알지 못하는 상황을 만들기

In [ ]:
## 1. 모듈 읽어오기
from langchain_core.runnables.history import RunnablePassthrough,RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

## 2. 환경변수
load_dotenv()

True

In [ ]:
## 3. 파일 읽기
file_name ='남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law_file = file.read()
    print(law_file[:50])


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [ ]:
## 4. prompt 생성

template = [
    (
        'system',
          '''
        당신은 육아휴직 법률 전문가입니다.
        아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
        답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
        - 문서: {law}
        '''),
        ('placeholder', '{chat_history}'),
        ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

In [ ]:
## 5. 인스턴스 생성

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm

ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x000001CBC8AABFA0>], client=<openai.resources.chat.completions.completions.Completions object at 0x000001CBC8BCA350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001CBC8BED0F0>, root_client=<openai.OpenAI object at 0x000001CBC6248C70>, root_async_client=<openai.AsyncOpenAI object at 0x000001CBC8BCA410>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True)

In [ ]:
## 6. chain 구성
chain = prompt | llm 

In [ ]:
## 7. 대화 이력 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

store = {}

In [ ]:
## 8.함수 정의: 대화 이력 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [ ]:
def trim_messages(messages, k=2):
    return messages[-k:]

In [ ]:
## 10. chain에 대화 이력 기능 래핑해서 추가

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

In [ ]:
## 11. 실행
with_message_history.invoke(
    {'law': law_file, 'query':query},
    config={'configurable': {'session_id': 'abc'}}
)


남편이 사용할 수 있는 육아휴직일 수는 법률상 명확하게 정해진 바가 있으며, 관련 내용은 「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」(남녀고용평등법) 제19조(육아휴직) 및 제19조의4(육아휴직의 사용형태)에서 확인할 수 있습니다.

⑴ **육아휴직 기간**은 원칙적으로 1년 이내로 하되, 법령에서는 남녀 모두가 동일하게 1년 이내의 육아휴직을 신청할 수 있도록 하고 있으며, 남편의 경우 특별히 연간 최대 사용 가능한 일수에 대한 제한은 별도로 명시되어 있지 않습니다. 다만, 남편이 육아휴직을 신청하는 경우, 육아휴직은「근로자」의 권리로서 법률상 인정되고, 여성이 사용하는 것과 동일한 조건으로 허용됩니다.

⑵ **육아휴직의 나눠 사용**은 「남녀고용평등법」 제19조의4(육아휴직과 육아휴직의 분할사용)에서 규정하고 있으며, 이 조문에 따라 근로자는 육아휴직을 최대 3회에 한정하여 나누어 사용할 수 있습니다. 이 때, 임신 중인 여성 근로자가 모성보호를 위해 사용한 육아휴직의 횟수는 포함되지 않습니다.

즉, **남편이 사용할 수 있는 육아휴직일 수는 최대 1년**이며, 이를 3회까지 나누어 사용할 수 있습니다. 단, 각 분할이 최소 1개월 이상이어야 하며, 여러 차례 사용 시에도 전체 기간이 1년을 초과하지 않도록 해야 합니다.

추가적으로, 육아휴직 기간 동안 해고 또는 불리한 처우를 하지 못하는 등 근로자의 권리를 보호하는 법적 규정도 마련되어 있습니다.

> **참고 법령: 제19조(육아휴직), 제19조의4(육아휴직의 사용형태)**

**요약:**  
남편이 사용할 수 있는 육아휴직일 수는 최대 1년이며, 3회에 나누어 사용할 수 있습니다.

AIMessage(content='남편이 사용할 수 있는 육아휴직일 수는 법률상 명확하게 정해진 바가 있으며, 관련 내용은 「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」(남녀고용평등법) 제19조(육아휴직) 및 제19조의4(육아휴직의 사용형태)에서 확인할 수 있습니다.\n\n⑴ **육아휴직 기간**은 원칙적으로 1년 이내로 하되, 법령에서는 남녀 모두가 동일하게 1년 이내의 육아휴직을 신청할 수 있도록 하고 있으며, 남편의 경우 특별히 연간 최대 사용 가능한 일수에 대한 제한은 별도로 명시되어 있지 않습니다. 다만, 남편이 육아휴직을 신청하는 경우, 육아휴직은「근로자」의 권리로서 법률상 인정되고, 여성이 사용하는 것과 동일한 조건으로 허용됩니다.\n\n⑵ **육아휴직의 나눠 사용**은 「남녀고용평등법」 제19조의4(육아휴직과 육아휴직의 분할사용)에서 규정하고 있으며, 이 조문에 따라 근로자는 육아휴직을 최대 3회에 한정하여 나누어 사용할 수 있습니다. 이 때, 임신 중인 여성 근로자가 모성보호를 위해 사용한 육아휴직의 횟수는 포함되지 않습니다.\n\n즉, **남편이 사용할 수 있는 육아휴직일 수는 최대 1년**이며, 이를 3회까지 나누어 사용할 수 있습니다. 단, 각 분할이 최소 1개월 이상이어야 하며, 여러 차례 사용 시에도 전체 기간이 1년을 초과하지 않도록 해야 합니다.\n\n추가적으로, 육아휴직 기간 동안 해고 또는 불리한 처우를 하지 못하는 등 근로자의 권리를 보호하는 법적 규정도 마련되어 있습니다.\n\n> **참고 법령: 제19조(육아휴직), 제19조의4(육아휴직의 사용형태)**\n\n**요약:**  \n남편이 사용할 수 있는 육아휴직일 수는 최대 1년이며, 3회에 나누어 사용할 수 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b3

In [ ]:
## 9. 사용자 질문 받기
query = input ('육아휴직과 관련된 질문하세요 >>')

In [ ]:
import openai
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "테스트 메시지"}
    ]
)

print(response.choices[0].message.content)




안녕하세요! 테스트 메시지를 받았습니다. 무엇을 도와드릴까요?


In [ ]:
import openai
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "테스트 메시지"}
    ]
)

print(response.choices[0].message.content)


안녕하세요! 테스트 메시지를 받았습니다. 어떻게 도와드릴까요?
